In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from multiprocessing import Pool
import datetime

import scrapers

In [2]:
import itertools


def p_imap_unordered_set(pool, func, iterable):
    temp = pool.imap_unordered(func, iterable)
    return set(itertools.chain.from_iterable(temp))


the_time = lambda: datetime.datetime.now().time().strftime('%H:%M:%S')

In [3]:
riders_done = set()
teams_done = set()
results = set()
teams_to_do = set()

print(the_time())

with Pool(100) as p:
    top_mens_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(1960, 2021), range(1, 4), [True])
    )
    top_womens_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(2018, 2021), range(1, 4), [False])
    )

top_riders = top_mens_riders | top_womens_riders

riders_to_do = top_riders.copy()
for i in range(50):
    if not riders_to_do:
        break
    print(f"{the_time()} - iter {i+1}, riders_to_do: {len(riders_to_do)}")
    with Pool(50) as p:
        fresh_results = p_imap_unordered_set(p, scrapers.scrape_rider, riders_to_do)

    results.update(fresh_results)
    riders_done.update(riders_to_do)

    teams_to_do = set(team for _, team in fresh_results) - teams_done

    print(f"{the_time()} - teams_to_do: {len(teams_to_do)}")

    with Pool(50) as p:
        riders_to_do = p_imap_unordered_set(p, scrapers.scrape_team, teams_to_do) - riders_done

    teams_done.update(teams_to_do)

print(the_time())

00:07:11
00:07:38 - iter 1, riders_to_do: 4762
00:10:12 - teams_to_do: 6312


IndexError: Problem with '4581'

In [ ]:
with open("top_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_riders)))
    
with open("top_mens_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_mens_riders)))
    
with open("top_womens_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_womens_riders)))

df = pd.DataFrame(iter(results), columns=["rider", "team"])
df.to_csv("scraped.csv", index=False)

In [ ]:
with requests.get(f"https://firstcycling.com/rider.php?r=32895&teams=1") as fp:
    soup = BeautifulSoup(fp.text, "html.parser")
tables = soup.find_all("table", **{"class": "tablesorter"})

In [ ]:
tables[1].find_all("a")